# INFO 3402 – Week 02: Aggregating and Summarizing

[Brian C. Keegan, Ph.D.](http://brianckeegan.com/)  
[Assistant Professor, Department of Information Science](https://www.colorado.edu/cmci/people/information-science/brian-c-keegan)  
University of Colorado Boulder  

Copyright and distributed under an [MIT License](https://opensource.org/licenses/MIT)  

In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 200
pd.options.display.float_format = '{:,}'.format

Groupby-aggregation is an *extremely* common and ***very*** powerful mechanism for summarizing data. Where we had combined different datasets together using joins and merges to add in additional (typically) columns, groupby-aggregation refers to summarizing (typically) rows of data.

Groupby-aggregation, as the name suggests, involves a sequence of different steps:

* **First**, grouping similar data together when they share a common value/identifier. Examples include schools within the same district, events on the same date, or actions taken by the same account.  
* **Second**, applying some function to each group independently to summarize the statistics about that group.
* **Third**, combining the results from each group back into a data structure comparing results across all groups.

Typically the second and third steps above are combined into an "aggregation" step. This figure summarizes the steps:

![split-apply-combine](https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png)

## Data

Load the data from the CSV file and use the "parse_dates" parameter to convert the "Week date" column into valid Timestamp objects.

In [2]:
# Load
deaths_df = pd.read_csv(
    'CDC_deaths_2014_2022.csv',
    parse_dates = ['Week date']
)

# Inspect
deaths_df.head()

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
0,Alabama,2014,1,2014-01-04,1057.0,992.0,26.0,196.0,29.0,38.0,38.0,66.0,17.0,21.0,25.0,264.0,51.0,0.0,0.0
1,Alabama,2014,2,2014-01-11,1077.0,991.0,30.0,203.0,25.0,46.0,37.0,81.0,18.0,23.0,35.0,250.0,49.0,0.0,0.0
2,Alabama,2014,3,2014-01-18,1048.0,975.0,21.0,210.0,34.0,31.0,34.0,77.0,19.0,24.0,19.0,243.0,60.0,0.0,0.0
3,Alabama,2014,4,2014-01-25,1022.0,967.0,25.0,205.0,23.0,25.0,42.0,74.0,16.0,22.0,35.0,249.0,53.0,0.0,0.0
4,Alabama,2014,5,2014-02-01,1041.0,954.0,18.0,200.0,26.0,38.0,26.0,69.0,15.0,18.0,24.0,269.0,50.0,0.0,0.0


### Mini-poll 1

What is the syntax to pull out just the columns for "Year", "Week", and "Alzheimer disease"?

In [7]:
column_list = ['Year','Week','Alzheimer disease']
deaths_df.loc[:,column_list]

,Year,Week,Alzheimer disease
0,2014,1,38.0
1,2014,2,46.0
2,2014,3,31.0
3,2014,4,25.0
4,2014,5,38.0
...,...,...,...
21731,2021,48,0.0
21732,2021,49,0.0
21733,2021,50,0.0
21734,2021,51,0.0


### Mini-poll 2

What is the syntax to pull out the rows corresponding with the "Year" 2014?

In [10]:
deaths_df[deaths_df['Year'] == 2014]

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
0,Alabama,2014,1,2014-01-04,1057.0,992.0,26.0,196.0,29.0,38.0,38.0,66.0,17.0,21.0,25.0,264.0,51.0,0.0,0.0
1,Alabama,2014,2,2014-01-11,1077.0,991.0,30.0,203.0,25.0,46.0,37.0,81.0,18.0,23.0,35.0,250.0,49.0,0.0,0.0
2,Alabama,2014,3,2014-01-18,1048.0,975.0,21.0,210.0,34.0,31.0,34.0,77.0,19.0,24.0,19.0,243.0,60.0,0.0,0.0
3,Alabama,2014,4,2014-01-25,1022.0,967.0,25.0,205.0,23.0,25.0,42.0,74.0,16.0,22.0,35.0,249.0,53.0,0.0,0.0
4,Alabama,2014,5,2014-02-01,1041.0,954.0,18.0,200.0,26.0,38.0,26.0,69.0,15.0,18.0,24.0,269.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21366,Wyoming,2014,49,2014-12-06,76.0,69.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0
21367,Wyoming,2014,50,2014-12-13,77.0,69.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
21368,Wyoming,2014,51,2014-12-20,85.0,80.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0
21369,Wyoming,2014,52,2014-12-27,83.0,74.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0


In [11]:
deaths_df.loc[deaths_df['Year'] == 2014]

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
0,Alabama,2014,1,2014-01-04,1057.0,992.0,26.0,196.0,29.0,38.0,38.0,66.0,17.0,21.0,25.0,264.0,51.0,0.0,0.0
1,Alabama,2014,2,2014-01-11,1077.0,991.0,30.0,203.0,25.0,46.0,37.0,81.0,18.0,23.0,35.0,250.0,49.0,0.0,0.0
2,Alabama,2014,3,2014-01-18,1048.0,975.0,21.0,210.0,34.0,31.0,34.0,77.0,19.0,24.0,19.0,243.0,60.0,0.0,0.0
3,Alabama,2014,4,2014-01-25,1022.0,967.0,25.0,205.0,23.0,25.0,42.0,74.0,16.0,22.0,35.0,249.0,53.0,0.0,0.0
4,Alabama,2014,5,2014-02-01,1041.0,954.0,18.0,200.0,26.0,38.0,26.0,69.0,15.0,18.0,24.0,269.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21366,Wyoming,2014,49,2014-12-06,76.0,69.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0
21367,Wyoming,2014,50,2014-12-13,77.0,69.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
21368,Wyoming,2014,51,2014-12-20,85.0,80.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0
21369,Wyoming,2014,52,2014-12-27,83.0,74.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0


### Repeated values

The values in some columns like "State", "Year", "Week", "Week date" are repeated across rows. Each row is the number of deaths in the state for that week.

Let's check to make sure these repeat in the same way using the `.value_counts()` method ([docs](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)). 

There are the 50 states plus two extras: the District of Columbia and Puerto Rico. So we coincidentally have both 52 "states" and 52 (ish) weeks in the year.

In [14]:
deaths_df['State'].value_counts()

Alabama                 418
Alaska                  418
Nevada                  418
New Hampshire           418
New Jersey              418
New Mexico              418
New York                418
North Carolina          418
North Dakota            418
Ohio                    418
Oklahoma                418
Oregon                  418
Pennsylvania            418
Puerto Rico             418
Rhode Island            418
South Carolina          418
South Dakota            418
Tennessee               418
Texas                   418
Utah                    418
Vermont                 418
Virginia                418
Washington              418
West Virginia           418
Wisconsin               418
Nebraska                418
Montana                 418
Missouri                418
Idaho                   418
Arizona                 418
Arkansas                418
California              418
Colorado                418
Connecticut             418
Delaware                418
District of Columbia

`.value_counts()` by default sorts the results in descending order. We can chain the `.sort_index()` method ([docs](https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_index.html)) to sort the index in ascending order.

There's 52 extra observations for the 2014 and 2020 years. Why might that be?

In [15]:
deaths_df['Year'].value_counts().sort_index()

2014    2756
2015    2704
2016    2704
2017    2704
2018    2704
2019    2704
2020    2756
2021    2704
Name: Year, dtype: int64

I suspect that's because there's a "53rd" week of the year. Sure enough there are 104 observations (52 states x 2 years) of a 53rd week.

In [16]:
deaths_df['Week'].value_counts().sort_index()

1     416
2     416
3     416
4     416
5     416
6     416
7     416
8     416
9     416
10    416
11    416
12    416
13    416
14    416
15    416
16    416
17    416
18    416
19    416
20    416
21    416
22    416
23    416
24    416
25    416
26    416
27    416
28    416
29    416
30    416
31    416
32    416
33    416
34    416
35    416
36    416
37    416
38    416
39    416
40    416
41    416
42    416
43    416
44    416
45    416
46    416
47    416
48    416
49    416
50    416
51    416
52    416
53    104
Name: Week, dtype: int64

This could happen when there's either a week ending date that is really early in January or really late in December and gets counted with another year.
 
Here I use the `.loc` selector to [Boolean index](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) to rows of data that have a 53rd week (`deaths_df['Week'] == 53`) and I simultaneously select the "Week date" and "Year" columns (`,['Week date','Year']`). `.loc` lets you specify both the rows and columns you want returned ([docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)). Then I count how many occurrences there are of these Week dates happening in the 53rd week with `.value_counts()`.

So there are weeks ending in 2015 and 2021 that technically belong to 2014 and 2020.

In [17]:
deaths_df.loc[
    deaths_df['Week'] == 53, # Select the rows that are 53rd weeks
    ['Week date','Year'] # Select the columns for Week date and Year
].value_counts() # Count the values

Week date   Year
2015-01-03  2014    52
2021-01-02  2020    52
dtype: int64

## Aggregating

### Building intuition
What were the total number of deaths per year from "Malignant neoplasms" (cancer)?

* **Splitting**: use boolean indexing make "baby DataFrames" for each different value of "Year"
* **Applying**: take the sum of the "Malignant neoplasm" column in each baby DataFrame (each year's state-weekly record of deaths)
* **Combining**: combine the results of the total deaths for each year back into a DataFrame

Let's start with splitting.

In [18]:
# Make a "baby DataFrame" for the year 2014 and the column "Malignant neoplasms"

cancer_2014 = deaths_df.loc[deaths_df['Year'] == 2014,'Malignant neoplasms']
cancer_2014

0        196.0
1        203.0
2        210.0
3        205.0
4        200.0
         ...  
21366     14.0
21367     18.0
21368     17.0
21369     18.0
21370     12.0
Name: Malignant neoplasms, Length: 2756, dtype: float64

There are 2756 rows of data. Let's do a quick calculation to make sure this matches our expectations.

2014 is one of the years with 53 weeks and there are 52 states in the data. So 52\*53 should hopefully equal 2756.

In [19]:
53*52

2756

Now we can apply a function. In this case, we want to total the number of deaths from cancer across all states and weeks in the year 2014. So we can just `.sum()` the `cancer_2014` Series. This is the total number of cancer deaths in 2014.

In [20]:
cancer_2014.sum()

593437.0

Now we'd have to repeat these steps for 2015, 2016 and so on to get their totals. This is the combining step where the splitting and applying for each year are integrated into an annual count.

In [21]:
# Make an empty container to store results
total_deaths_d = {}

# Loop through the year values of 2014 to 2021
for year in range(2014,2022):
    
    # Make the baby DataFrame for cancer deaths in the year
    cancer_deaths = deaths_df.loc[deaths_df['Year'] == year,'Malignant neoplasms']
    
    # Add up those values
    total_cancer_deaths = cancer_deaths.sum()
    
    # Store the total value back in the dictionary
    total_deaths_d[year] = total_cancer_deaths
    
# Convert the dictionary to a pandas Series
total_deaths_s = pd.Series(total_deaths_d,name='Malignant neoplasms')

# Inspect
total_deaths_s

2014    593437.0
2015    585841.0
2016    586523.0
2017    589451.0
2018    589759.0
2019    590798.0
2020    604846.0
2021    578974.0
Name: Malignant neoplasms, dtype: float64

## Groupby-aggregation

That involved a lot of steps—remember that programmers are, first and foremost: lazy! 

The brilliance of groupby-aggregation is we can accomplish the same thing in a single line, which both saves us time and reduces the chances of errors!

You will almost always use groupby in combination with aggregation. The groupby by itself returns a data structure and the aggregation only works on groupby objects. So we string them together in the pattern of `df.groupby().agg()`.

In [23]:
deaths_df.groupby(['Year']).agg({'Malignant neoplasms':'sum'})

,Malignant neoplasms
Year,
2014,593437.0
2015,585841.0
2016,586523.0
2017,589451.0
2018,589759.0
2019,590798.0
2020,604846.0
2021,578974.0


### Splitting
Let's take that step-by-step, starting with the [`.groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) operation that splits the data. The most important concept you need to internalize is that when you do a `groupby` you are making a "baby DataFrame" where the rows of that DataFrame have something all in common: whatever value you grouped on. 

You saw above how we used Boolean indexing to make a baby DataFrame for each year. I will show two other ways that groupbys create "baby DataFrames" that share some value in common.

Create a `deaths_gb_year` that stores the results of grouping `deaths_df` by "Year".

In [24]:
deaths_gb_year = deaths_df.groupby(['Year'])

What is this groupby object? Not a whole lot to look at.

In [25]:
deaths_gb_year

In [30]:
deaths_gb_year.get_group(2021)

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
366,Alabama,2021,1,2021-01-09,1808.0,1706.0,21.0,223.0,36.0,70.0,19.0,79.0,27.0,24.0,33.0,336.0,62.0,546.0,527.0
367,Alabama,2021,2,2021-01-16,1831.0,1741.0,16.0,211.0,47.0,66.0,25.0,62.0,22.0,24.0,28.0,346.0,67.0,595.0,569.0
368,Alabama,2021,3,2021-01-23,1790.0,1688.0,16.0,194.0,39.0,63.0,19.0,64.0,26.0,33.0,28.0,336.0,63.0,579.0,546.0
369,Alabama,2021,4,2021-01-30,1637.0,1559.0,20.0,206.0,37.0,63.0,20.0,74.0,24.0,22.0,21.0,311.0,63.0,490.0,460.0
370,Alabama,2021,5,2021-02-06,1508.0,1409.0,22.0,216.0,24.0,66.0,23.0,69.0,19.0,22.0,26.0,302.0,62.0,345.0,332.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,Wyoming,2021,48,2021-12-04,105.0,98.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,20.0,17.0
21732,Wyoming,2021,49,2021-12-11,117.0,114.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,23.0,22.0
21733,Wyoming,2021,50,2021-12-18,89.0,83.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,16.0,15.0
21734,Wyoming,2021,51,2021-12-25,94.0,93.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,15.0,15.0


## Applying

Now that we have different baby DataFrames that share a value in common (Year), we want to apply some function that reduces all the data down to a single summary value for the entire DataFrame. In other words, we're summarizing all the values of the baby DataFrame down to a single value.

In the simple example of sum "Malignant neoplasms" for "Year" we're adding up the 2756 rows of data for states' weekly numbers into a single number: the total number of cancer deaths across all states and weeks in that year.

In [31]:
deaths_gb_year.get_group(2014)['Malignant neoplasms'].sum()

593437.0

We could do this for each year.

In [32]:
for year in range(2014,2022):
    print(year, deaths_gb_year.get_group(year)['Malignant neoplasms'].sum())

2014 593437.0
2015 585841.0
2016 586523.0
2017 589451.0
2018 589759.0
2019 590798.0
2020 604846.0
2021 578974.0


We can apply just about any numeric aggregation function. Some common ones:

* `len` - Number of rows in the baby DataFrame
* `mean` - Average value of a column in the baby DataFrame
* `min` and `max` - Minimum and maximum values of a column in the baby DataFrame
* `sum` - Total value of a column in the baby DataFrame
* `first` and `last` - First and last value of a column in the baby DataFrame

You can pass some of these functions as a string (`'mean'`) and others as a function (`np.mean`). 

You can use `apply` on column in the GroupBy object and pass a function to summarize all the values in the baby DataFrame. This returns the annual total counts of cancer deaths by adding up all the values in each of the annual baby DataFrames. 

You only use `apply` on a GroupBy object in rare and advanced cases: **99.999% of the time you will use `groupby` in combination with `aggregate`.**

In [33]:
deaths_gb_year['Malignant neoplasms'].apply(np.sum)

Year
2014    593437.0
2015    585841.0
2016    586523.0
2017    589451.0
2018    589759.0
2019    590798.0
2020    604846.0
2021    578974.0
Name: Malignant neoplasms, dtype: float64

## Combining

The [`.agg`](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.agg.html) function serves to both apply and combine the results on a GroupBy object. There are several strategies for using `.agg` on a GroupBy object.

First, you can access a column in a GroupBy object and pass the `.agg` method with the function(s) you want it to aggregate.

In [35]:
deaths_df.groupby(['Year']).agg({'Diabetes mellitus':'sum'})

,Diabetes mellitus
Year,
2014,75953.0
2015,76647.0
2016,77604.0
2017,81367.0
2018,82595.0
2019,85451.0
2020,101653.0
2021,95952.0


In [36]:
deaths_gb_year['Diabetes mellitus'].agg('sum')

Year
2014     75953.0
2015     76647.0
2016     77604.0
2017     81367.0
2018     82595.0
2019     85451.0
2020    101653.0
2021     95952.0
Name: Diabetes mellitus, dtype: float64

You can also pass a list of aggregation functions. We'll use the pre-defined aggregation values in pandas:

* `'mean'` - average deaths per state and week across the year
* `'min'` - minimum deaths per state and week across the year
* `'max'` - maximum deaths per state and week across the year
* `'count'` - number of rows per year
* `'sum'` - total deaths per year

### Mini-poll 3

What would be the syntax for the total number of heart disease deaths in Colorado from 2014 to the present?

In [40]:
deaths_df.groupby(['State']).agg({'Diseases of heart':np.sum}).loc['Colorado']

Diseases of heart    60472.0
Name: Colorado, dtype: float64

## Multiple aggregation functions

In [41]:
deaths_gb_year['Diabetes mellitus'].agg(['mean','min','max','count','sum'])

,mean,min,max,count,sum
Year,,,,,
2014,27.559144,0.0,199.0,2756,75953.0
2015,28.345784,0.0,215.0,2704,76647.0
2016,28.699704,0.0,222.0,2704,77604.0
2017,30.091346,0.0,253.0,2704,81367.0
2018,30.545488,0.0,285.0,2704,82595.0
2019,31.601701,0.0,248.0,2704,85451.0
2020,36.884253,0.0,380.0,2756,101653.0
2021,35.485207,0.0,356.0,2704,95952.0


You could also pass a list of functions, which is probably more safe in general. Note how the name of the function becomes the name of the column.

In [42]:
deaths_gb_year['Diabetes mellitus'].agg([np.mean,np.min,np.max,len,np.sum])

,mean,amin,amax,len,sum
Year,,,,,
2014,27.559144,0.0,199.0,2756,75953.0
2015,28.345784,0.0,215.0,2704,76647.0
2016,28.699704,0.0,222.0,2704,77604.0
2017,30.091346,0.0,253.0,2704,81367.0
2018,30.545488,0.0,285.0,2704,82595.0
2019,31.601701,0.0,248.0,2704,85451.0
2020,36.884253,0.0,380.0,2756,101653.0
2021,35.485207,0.0,356.0,2704,95952.0


The first option only works if you want to apply-combine the values for a single column. The second option is to create an "aggregation dictionary". The keys in the dictionary are the names of the columns to perform an aggregation on and the values of the dictionary are the function(s) to apply.

Replicating the aggregation we've already done:

In [45]:
# Define the aggregation dictionary
agg_d = {'Diabetes mellitus':[np.mean,np.min,np.max,len,np.sum],'Diseases of heart':['sum','mean']}

# Perform the aggregation on the groupby using the dictionary
deaths_gb_year.agg(agg_d)

Diabetes mellitus                             Diseases of heart  \
                  mean amin   amax   len       sum               sum   
Year                                                                   
2014         27.559144  0.0  199.0  2756   75953.0          614870.0   
2015         28.345784  0.0  215.0  2704   76647.0          619126.0   
2016         28.699704  0.0  222.0  2704   77604.0          619807.0   
2017         30.091346  0.0  253.0  2704   81367.0          633482.0   
2018         30.545488  0.0  285.0  2704   82595.0          641349.0   
2019         31.601701  0.0  248.0  2704   85451.0          644496.0   
2020         36.884253  0.0  380.0  2756  101653.0          692775.0   
2021         35.485207  0.0  356.0  2704   95952.0          649433.0   

                  
            mean  
Year              
2014  223.102322  
2015  228.966716  
2016  229.218565  
2017  234.275888  
2018  237.185281  
2019  238.349112  
2020  251.369739  
2021  240.174926

The aggregation dictionary is valuable when you want to apply functions to multiple columns and combine all the results together. This returns a MultiIndex on the columns.

I'm going to save the DataFrame as `immunizations_agg_df` to access for other examples. 

In [46]:
# Define the aggregation dictionary
agg_d = {'Diabetes mellitus':[np.mean,min,max,len,np.sum],
         'Septicemia':[np.mean,np.median,min,max,np.sum]}

# Perform the aggregation on the groupby using the dictionary
deaths_agg_year = deaths_gb_year.agg(agg_d)
deaths_agg_year

Diabetes mellitus                             Septicemia              \
                  mean  min    max   len       sum       mean median  min   
Year                                                                        
2014         27.559144  0.0  199.0  2756   75953.0  12.835994   10.0  0.0   
2015         28.345784  0.0  215.0  2704   76647.0  13.298447   10.0  0.0   
2016         28.699704  0.0  222.0  2704   77604.0  13.236686   10.0  0.0   
2017         30.091346  0.0  253.0  2704   81367.0  13.468565   10.0  0.0   
2018         30.545488  0.0  285.0  2704   82595.0  13.321006    0.0  0.0   
2019         31.601701  0.0  248.0  2704   85451.0  12.487056    0.0  0.0   
2020         36.884253  0.0  380.0  2756  101653.0  13.002177   10.0  0.0   
2021         35.485207  0.0  356.0  2704   95952.0  12.675666    0.0  0.0   

                      
        max      sum  
Year                  
2014  111.0  35376.0  
2015  109.0  35959.0  
2016  114.0  35792.0  
2017  121.0  36419.0  
2018  133.0  36020.0  
2019   76.0  33765.0  
2020   85.0  35834.0  
2021   94.0  34275.0

We can access a Series from the MultiIndexed columns in `immunizations_agg_df` by passing a tuple of the column names: `('Diabetes mellitus','sum')`.

In [47]:
deaths_agg_year[('Septicemia','sum')]

Year
2014    35376.0
2015    35959.0
2016    35792.0
2017    36419.0
2018    36020.0
2019    33765.0
2020    35834.0
2021    34275.0
Name: (Septicemia, sum), dtype: float64

You can also drop the MultiIndex ([docs](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.droplevel.html)) to have a simple index and relabel the columns.

In [48]:
# Drop the top level of the MultiIndex on the columns
deaths_agg_year.columns = deaths_agg_year.columns.droplevel(0)

# Relabel the columns
deaths_agg_year.columns = [
    'mean_diabetes',
    'min_diabetes',
    'max_diabetes',
    'count_diabetes',
    'total_diabetes',
    'mean_septicemia',
    'median_septicemia',
    'min_septicemia',
    'max_septicemia',
    'total_septicemia'
]

# Inspect
deaths_agg_year

,mean_diabetes,min_diabetes,max_diabetes,count_diabetes,total_diabetes,mean_septicemia,median_septicemia,min_septicemia,max_septicemia,total_septicemia
Year,,,,,,,,,,
2014,27.559144,0.0,199.0,2756,75953.0,12.835994,10.0,0.0,111.0,35376.0
2015,28.345784,0.0,215.0,2704,76647.0,13.298447,10.0,0.0,109.0,35959.0
2016,28.699704,0.0,222.0,2704,77604.0,13.236686,10.0,0.0,114.0,35792.0
2017,30.091346,0.0,253.0,2704,81367.0,13.468565,10.0,0.0,121.0,36419.0
2018,30.545488,0.0,285.0,2704,82595.0,13.321006,0.0,0.0,133.0,36020.0
2019,31.601701,0.0,248.0,2704,85451.0,12.487056,0.0,0.0,76.0,33765.0
2020,36.884253,0.0,380.0,2756,101653.0,13.002177,10.0,0.0,85.0,35834.0
2021,35.485207,0.0,356.0,2704,95952.0,12.675666,0.0,0.0,94.0,34275.0


## Grouping by multiple values

You can group on multiple values by passing them as a list to `groupby`.

Let's play with all the disease data now. Slice the `deaths_df` columns from 6 onward to get the columns with different disease types.

In [52]:
deaths_df

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
0,Alabama,2014,1,2014-01-04,1057.0,992.0,26.0,196.0,29.0,38.0,38.0,66.0,17.0,21.0,25.0,264.0,51.0,0.0,0.0
1,Alabama,2014,2,2014-01-11,1077.0,991.0,30.0,203.0,25.0,46.0,37.0,81.0,18.0,23.0,35.0,250.0,49.0,0.0,0.0
2,Alabama,2014,3,2014-01-18,1048.0,975.0,21.0,210.0,34.0,31.0,34.0,77.0,19.0,24.0,19.0,243.0,60.0,0.0,0.0
3,Alabama,2014,4,2014-01-25,1022.0,967.0,25.0,205.0,23.0,25.0,42.0,74.0,16.0,22.0,35.0,249.0,53.0,0.0,0.0
4,Alabama,2014,5,2014-02-01,1041.0,954.0,18.0,200.0,26.0,38.0,26.0,69.0,15.0,18.0,24.0,269.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,Wyoming,2021,48,2021-12-04,105.0,98.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,20.0,17.0
21732,Wyoming,2021,49,2021-12-11,117.0,114.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,23.0,22.0
21733,Wyoming,2021,50,2021-12-18,89.0,83.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,16.0,15.0
21734,Wyoming,2021,51,2021-12-25,94.0,93.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,15.0,15.0


In [49]:
disease_cols = deaths_df.columns[6:]

disease_cols

Index(['Septicemia', 'Malignant neoplasms', 'Diabetes mellitus',
       'Alzheimer disease', 'Influenza and pneumonia',
       'Chronic lower respiratory diseases',
       'Other diseases of respiratory system',
       'Nephritis, nephrotic syndrome and nephrosis', 'Other',
       'Diseases of heart', 'Cerebrovascular diseases',
       'COVID-19, Multiple Cause', 'COVID-19, Underlying Cause'],
      dtype='object')

We're going to group on "Year" then on "State" to get the annnual state-level counts of disease.

In [50]:
# Do the groupby
deaths_gb_year_state = deaths_df.groupby(['Year','State'])

Create an aggregation dictionary of the disease column names from `disease_cols` and use the "sum" as an aggregation function for each column.

In [51]:
# Define an aggregation dictionary for exemption rate columns
disease_agg_d = {col:'sum' for col in disease_cols}

disease_agg_d

{'Septicemia': 'sum',
 'Malignant neoplasms': 'sum',
 'Diabetes mellitus': 'sum',
 'Alzheimer disease': 'sum',
 'Influenza and pneumonia': 'sum',
 'Chronic lower respiratory diseases': 'sum',
 'Other diseases of respiratory system': 'sum',
 'Nephritis, nephrotic syndrome and nephrosis': 'sum',
 'Other': 'sum',
 'Diseases of heart': 'sum',
 'Cerebrovascular diseases': 'sum',
 'COVID-19, Multiple Cause': 'sum',
 'COVID-19, Underlying Cause': 'sum'}

Perform and inspect the aggregation.

In [53]:
# Perform the aggregation
deaths_gb_year_state_agg = deaths_gb_year_state.agg(disease_agg_d)

# Inspect
deaths_gb_year_state_agg

Septicemia  Malignant neoplasms  Diabetes mellitus  \
Year State                                                               
2014 Alabama            1031.0              10245.0             1278.0   
     Alaska                0.0                907.0                0.0   
     Arizona             169.0              11726.0             1990.0   
     Arkansas            354.0               6612.0              819.0   
     California         1302.0              59598.0             8397.0   
...                        ...                  ...                ...   
2021 Virginia           1019.0              15171.0             2509.0   
     Washington          369.0              13254.0             2141.0   
     West Virginia        43.0               3901.0              903.0   
     Wisconsin           440.0              11083.0             1631.0   
     Wyoming               0.0               1021.0                0.0   

                    Alzheimer disease  Influenza and pneumonia  \
Year State                                                       
2014 Alabama                   1870.0                   1004.0   
     Alaska                       0.0                      0.0   
     Arizona                   2557.0                    685.0   
     Arkansas                  1211.0                    600.0   
     California               12896.0                   6127.0   
...                               ...                      ...   
2021 Virginia                  2515.0                    878.0   
     Washington                3539.0                    318.0   
     West Virginia              665.0                    159.0   
     Wisconsin                 2343.0                    358.0   
     Wyoming                     10.0                      0.0   

                    Chronic lower respiratory diseases  \
Year State                                               
2014 Alabama                                    3072.0   
     Alaska                                        0.0   
     Arizona                                    3503.0   
     Arkansas                                   2149.0   
     California                                13052.0   
...                                                ...   
2021 Virginia                                   3087.0   
     Washington                                 2541.0   
     West Virginia                              1233.0   
     Wisconsin                                  2378.0   
     Wyoming                                      76.0   

                    Other diseases of respiratory system  \
Year State                                                 
2014 Alabama                                       799.0   
     Alaska                                          0.0   
     Arizona                                       409.0   
     Arkansas                                      156.0   
     California                                   2575.0   
...                                                  ...   
2021 Virginia                                     1302.0   
     Washington                                    805.0   
     West Virginia                                  26.0   
     Wisconsin                                     575.0   
     Wyoming                                         0.0   

                    Nephritis, nephrotic syndrome and nephrosis   Other  \
Year State                                                                
2014 Alabama                                              985.0  1244.0   
     Alaska                                                 0.0     0.0   
     Arizona                                               43.0  1174.0   
     Arkansas                                             583.0   192.0   
     California                                          3167.0  1169.0   
...                                                         ...     ...   
2021 Virginia                                            1568.

The index is a MultiIndex ([docs](https://pandas.pydata.org/docs/user_guide/advanced.html)) comprised of tuples of `(Year,State)`. We can access any row by passing the tuple to the `.loc` method.

This is the deaths in Colorado in 2021. COVID was the leading cause of death, greater than either cancer or heart disease.

In [56]:
deaths_gb_year_state_agg.loc[(2021,'Colorado')]

Septicemia                                      266.0
Malignant neoplasms                            7923.0
Diabetes mellitus                              1152.0
Alzheimer disease                              1756.0
Influenza and pneumonia                          20.0
Chronic lower respiratory diseases             2241.0
Other diseases of respiratory system            627.0
Nephritis, nephrotic syndrome and nephrosis     379.0
Other                                           985.0
Diseases of heart                              7908.0
Cerebrovascular diseases                       2051.0
COVID-19, Multiple Cause                       5803.0
COVID-19, Underlying Cause                     5073.0
Name: (2021, Colorado), dtype: float64

### Mini-poll 4

Using deaths_gb_year_state_agg, what is the number of deaths for all diseases in Massachusetts in 2016?

### Accessing levels

We can also access one level of the tuple.

These are all the state-level deaths in 2021.

In [58]:
deaths_gb_year_state_agg.loc[2014]

,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
State,,,,,,,,,,,,,
Alabama,1031.0,10245.0,1278.0,1870.0,1004.0,3072.0,799.0,985.0,1244.0,12532.0,2627.0,0.0,0.0
Alaska,0.0,907.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,779.0,0.0,0.0,0.0
Arizona,169.0,11726.0,1990.0,2557.0,685.0,3503.0,409.0,43.0,1174.0,11153.0,2277.0,0.0,0.0
Arkansas,354.0,6612.0,819.0,1211.0,600.0,2149.0,156.0,583.0,192.0,7583.0,1563.0,0.0,0.0
California,1302.0,59598.0,8397.0,12896.0,6127.0,13052.0,2575.0,3167.0,1169.0,59239.0,13989.0,0.0,0.0
Colorado,208.0,7572.0,829.0,1420.0,637.0,2504.0,297.0,260.0,148.0,7168.0,1776.0,0.0,0.0
Connecticut,432.0,6787.0,647.0,929.0,523.0,1398.0,395.0,469.0,78.0,7118.0,1317.0,0.0,0.0
Delaware,0.0,2000.0,21.0,0.0,0.0,269.0,0.0,10.0,0.0,1930.0,250.0,0.0,0.0
District of Columbia,0.0,1304.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1478.0,45.0,0.0,0.0


We could also access the other level of the MultiIndex. This requires a helper called IndexSlice ([docs](https://pandas.pydata.org/docs/reference/api/pandas.IndexSlice.html)), but is ***super*** powerful. We will use the `.loc` method in combination with [slicing with labels](https://pandas.pydata.org/docs/user_guide/indexing.html#slicing-with-labels). The first colon inside `idx` selects all the years, the "Colorado" after the comma only selects the Colorado states, and the colon outside the `idx` bracket selects all the columns.

These are all the deaths in Colorado across years.

In [61]:
idx = pd.IndexSlice

deaths_gb_year_state_agg.loc[idx[:,'Colorado'],:]

,,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
Year,State,,,,,,,,,,,,,
2014,Colorado,208.0,7572.0,829.0,1420.0,637.0,2504.0,297.0,260.0,148.0,7168.0,1776.0,0.0,0.0
2015,Colorado,283.0,7648.0,871.0,1598.0,532.0,2559.0,360.0,260.0,43.0,7108.0,1894.0,0.0,0.0
2016,Colorado,295.0,7962.0,929.0,1840.0,397.0,2583.0,359.0,274.0,11.0,7399.0,1951.0,0.0,0.0
2017,Colorado,311.0,7841.0,1027.0,1836.0,447.0,2596.0,453.0,358.0,0.0,7209.0,2022.0,0.0,0.0
2018,Colorado,294.0,7869.0,987.0,1655.0,361.0,2652.0,490.0,252.0,30.0,7495.0,2046.0,0.0,0.0
2019,Colorado,212.0,8053.0,1051.0,1919.0,281.0,2525.0,489.0,323.0,0.0,7906.0,2049.0,0.0,0.0
2020,Colorado,234.0,8421.0,1183.0,2197.0,264.0,2541.0,519.0,307.0,20.0,8279.0,2276.0,5133.0,4470.0
2021,Colorado,266.0,7923.0,1152.0,1756.0,20.0,2241.0,627.0,379.0,985.0,7908.0,2051.0,5803.0,5073.0


In [62]:
deaths_gb_year_state_agg.loc[idx[:,'California'],['Cerebrovascular diseases']]

,,Cerebrovascular diseases
Year,State,
2014,California,13989.0
2015,California,15008.0
2016,California,15553.0
2017,California,16287.0
2018,California,16416.0
2019,California,16756.0
2020,California,18245.0
2021,California,17824.0


## Pivot tables

You may have encountered pivot tables in spreadsheet applications and pandas has a similar function. A pivot table reshapes the data to have specific rows, columns, and values.

Let's make a pivot table of the "All Cause" deaths as values, the year as columns, and the states as rows. We specify an aggregation function ("aggfunc") to summarize the values like we do for groupby-aggregations. In this case, this is the sumer of each weekly "All Cause" value for a state in a year.

In [63]:
deaths_df

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
0,Alabama,2014,1,2014-01-04,1057.0,992.0,26.0,196.0,29.0,38.0,38.0,66.0,17.0,21.0,25.0,264.0,51.0,0.0,0.0
1,Alabama,2014,2,2014-01-11,1077.0,991.0,30.0,203.0,25.0,46.0,37.0,81.0,18.0,23.0,35.0,250.0,49.0,0.0,0.0
2,Alabama,2014,3,2014-01-18,1048.0,975.0,21.0,210.0,34.0,31.0,34.0,77.0,19.0,24.0,19.0,243.0,60.0,0.0,0.0
3,Alabama,2014,4,2014-01-25,1022.0,967.0,25.0,205.0,23.0,25.0,42.0,74.0,16.0,22.0,35.0,249.0,53.0,0.0,0.0
4,Alabama,2014,5,2014-02-01,1041.0,954.0,18.0,200.0,26.0,38.0,26.0,69.0,15.0,18.0,24.0,269.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,Wyoming,2021,48,2021-12-04,105.0,98.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,20.0,17.0
21732,Wyoming,2021,49,2021-12-11,117.0,114.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,23.0,22.0
21733,Wyoming,2021,50,2021-12-18,89.0,83.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,16.0,15.0
21734,Wyoming,2021,51,2021-12-25,94.0,93.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,15.0,15.0


In [66]:
allcause_pivot = pd.pivot_table(
    data = deaths_df,
    index = "State",
    columns = "Year",
    values = "All Cause",
    aggfunc = "sum"
)

allcause_pivot.sort_index(axis=1,ascending=False)

Year,2021,2020,2019,2018,2017,2016,2015,2014
State,,,,,,,,
Alabama,64993.0,64354.0,53057.0,53146.0,52132.0,51130.0,50661.0,50229.0
Alaska,5473.0,5076.0,4503.0,4289.0,4255.0,4305.0,4170.0,4081.0
Arizona,80620.0,78255.0,60450.0,59495.0,57885.0,56583.0,54382.0,52761.0
Arkansas,38931.0,38399.0,32183.0,31744.0,31707.0,30847.0,30830.0,30508.0
California,322673.0,326503.0,268775.0,268145.0,267106.0,260595.0,258512.0,250552.0
Colorado,47568.0,48256.0,39922.0,39007.0,38593.0,37878.0,36729.0,36474.0
Connecticut,32303.0,38644.0,31789.0,31333.0,31417.0,30439.0,30460.0,30392.0
Delaware,10880.0,11147.0,9222.0,9449.0,9087.0,8937.0,8524.0,8443.0
District of Columbia,6488.0,7556.0,5962.0,6127.0,6118.0,5976.0,5647.0,5568.0


You can also make fancy MultiIndexed pivot tables by passing lists of columns. Here we make a pivot table with the lower respiratory diseases as values, the index both the State and the Year, and the columns the week of the year.

In [65]:
lower_respiratory_pivot = pd.pivot_table(
    data = deaths_df,
    index = ["State","Year"],
    columns = "Week",
    values = "Chronic lower respiratory diseases",
    aggfunc = "sum"
)

# Show Colorado
lower_respiratory_pivot.loc['Colorado']

Week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014,53.0,59.0,47.0,53.0,44.0,51.0,65.0,50.0,49.0,47.0,51.0,56.0,58.0,49.0,58.0,45.0,48.0,41.0,47.0,39.0,46.0,46.0,37.0,44.0,42.0,46.0,42.0,39.0,38.0,32.0,40.0,39.0,42.0,39.0,41.0,56.0,42.0,42.0,40.0,44.0,53.0,42.0,35.0,40.0,47.0,44.0,49.0,41.0,46.0,52.0,52.0,68.0,88.0
2015,65.0,55.0,57.0,54.0,54.0,59.0,51.0,56.0,53.0,58.0,54.0,50.0,54.0,45.0,66.0,54.0,55.0,46.0,42.0,54.0,49.0,41.0,39.0,57.0,63.0,39.0,30.0,42.0,43.0,49.0,46.0,50.0,36.0,43.0,45.0,47.0,40.0,40.0,45.0,38.0,46.0,48.0,38.0,61.0,48.0,57.0,58.0,36.0,59.0,61.0,45.0,38.0,NaN
2016,62.0,64.0,56.0,51.0,54.0,58.0,69.0,54.0,58.0,62.0,52.0,68.0,72.0,47.0,61.0,42.0,47.0,51.0,60.0,56.0,57.0,32.0,52.0,38.0,36.0,41.0,40.0,42.0,44.0,36.0,40.0,49.0,48.0,43.0,53.0,40.0,46.0,41.0,41.0,41.0,43.0,48.0,48.0,42.0,38.0,47.0,58.0,44.0,58.0,52.0,51.0,50.0,NaN
2017,63.0,67.0,62.0,53.0,66.0,62.0,58.0,71.0,56.0,66.0,65.0,51.0,60.0,51.0,48.0,52.0,42.0,66.0,59.0,60.0,55.0,37.0,48.0,39.0,33.0,47.0,52.0,43.0,42.0,36.0,47.0,42.0,40.0,44.0,39.0,42.0,52.0,38.0,35.0,44.0,33.0,40.0,42.0,43.0,46.0,45.0,41.0,37.0,47.0,52.0,61.0,76.0,NaN
2018,68.0,70.0,58.0,65.0,65.0,67.0,70.0,45.0,48.0,62.0,53.0,66.0,56.0,69.0,63.0,58.0,62.0,43.0,57.0,51.0,54.0,56.0,40.0,46.0,53.0,46.0,29.0,50.0,43.0,47.0,50.0,39.0,38.0,37.0,47.0,37.0,45.0,46.0,36.0,42.0,48.0,42.0,44.0,39.0,45.0,58.0,59.0,56.0,41.0,43.0,45.0,55.0,NaN
2019,42.0,57.0,57.0,54.0,55.0,44.0,57.0,56.0,57.0,61.0,67.0,44.0,63.0,47.0,68.0,62.0,42.0,37.0,40.0,56.0,48.0,44.0,34.0,62.0,48.0,60.0,50.0,43.0,29.0,44.0,48.0,41.0,40.0,33.0,47.0,45.0,35.0,39.0,42.0,41.0,45.0,48.0,56.0,55.0,43.0,42.0,54.0,47.0,48.0,53.0,52.0,43.0,NaN
2020,53.0,48.0,61.0,60.0,47.0,59.0,52.0,49.0,67.0,57.0,54.0,60.0,58.0,52.0,50.0,65.0,55.0,43.0,36.0,50.0,36.0,47.0,44.0,49.0,48.0,40.0,29.0,42.0,42.0,36.0,41.0,45.0,45.0,50.0,40.0,35.0,42.0,35.0,50.0,38.0,38.0,40.0,50.0,46.0,53.0,59.0,57.0,58.0,63.0,46.0,32.0,39.0,50.0
2021,46.0,36.0,39.0,41.0,64.0,48.0,52.0,48.0,35.0,38.0,44.0,43.0,48.0,40.0,30.0,38.0,50.0,45.0,40.0,42.0,48.0,33.0,45.0,37.0,42.0,34.0,36.0,45.0,43.0,43.0,49.0,30.0,38.0,35.0,49.0,45.0,42.0,38.0,40.0,48.0,49.0,49.0,45.0,49.0,55.0,40.0,48.0,52.0,44.0,54.0,37.0,32.0,NaN


## Exercises

### Exercise 1: Annual "All Cause" deaths

Perform a groupby-aggregation to compute the total "All Cause" of death by year. How much higher were deaths in 2020 than 2019?

In [3]:
deaths_df.head()

,State,Year,Week,Week date,All Cause,Natural Cause,Septicemia,Malignant neoplasms,Diabetes mellitus,Alzheimer disease,Influenza and pneumonia,Chronic lower respiratory diseases,Other diseases of respiratory system,"Nephritis, nephrotic syndrome and nephrosis",Other,Diseases of heart,Cerebrovascular diseases,"COVID-19, Multiple Cause","COVID-19, Underlying Cause"
0,Alabama,2014,1,2014-01-04,1057.0,992.0,26.0,196.0,29.0,38.0,38.0,66.0,17.0,21.0,25.0,264.0,51.0,0.0,0.0
1,Alabama,2014,2,2014-01-11,1077.0,991.0,30.0,203.0,25.0,46.0,37.0,81.0,18.0,23.0,35.0,250.0,49.0,0.0,0.0
2,Alabama,2014,3,2014-01-18,1048.0,975.0,21.0,210.0,34.0,31.0,34.0,77.0,19.0,24.0,19.0,243.0,60.0,0.0,0.0
3,Alabama,2014,4,2014-01-25,1022.0,967.0,25.0,205.0,23.0,25.0,42.0,74.0,16.0,22.0,35.0,249.0,53.0,0.0,0.0
4,Alabama,2014,5,2014-02-01,1041.0,954.0,18.0,200.0,26.0,38.0,26.0,69.0,15.0,18.0,24.0,269.0,50.0,0.0,0.0


In [5]:
all_cause_deaths_agg = deaths_df.groupby('Year').agg({'All Cause':'sum'})
all_cause_deaths_agg

,All Cause
Year,
2014,2652347.0
2015,2676162.0
2016,2704038.0
2017,2781306.0
2018,2806270.0
2019,2821219.0
2020,3389475.0
2021,3295886.0


In [11]:
3389475 - 2821219

568256

In [13]:
all_cause_deaths_agg.loc[2020,'All Cause']

3389475.0

In [14]:
all_cause_deaths_agg.loc[2019,'All Cause']

2821219.0

In [15]:
all_cause_deaths_agg.loc[2020,'All Cause'] - all_cause_deaths_agg.loc[2019,'All Cause']

568256.0

In [10]:
all_cause_deaths_agg.style.format(precision=0,thousands=',')

,All Cause
Year,
2014,"2,652,347"
2015,"2,676,162"
2016,"2,704,038"
2017,"2,781,306"
2018,"2,806,270"
2019,"2,821,219"
2020,"3,389,475"
2021,"3,295,886"


### Exercise 2: Weekly flu and pneumonia patterns

Make a pivot table with the Week as an index, Year as columns, and the total flu and pneumonia death. What is the deadliest week of the year on average for flu and pneumonia?

In [19]:
flu_pivot = pd.pivot_table(
    data = deaths_df,
    index = "Week",
    columns = "Year",
    values = "Influenza and pneumonia",
    aggfunc = "sum"
)

flu_pivot.head()

Year,2014,2015,2016,2017,2018,2019,2020,2021
Week,,,,,,,,
1,1543.0,2534.0,1144.0,1433.0,2433.0,1167.0,1435.0,894.0
2,1821.0,2307.0,1117.0,1495.0,2830.0,1220.0,1428.0,908.0
3,1820.0,2010.0,1044.0,1483.0,2845.0,1274.0,1380.0,849.0
4,1660.0,1787.0,1089.0,1438.0,2577.0,1153.0,1397.0,791.0
5,1540.0,1659.0,1078.0,1533.0,2385.0,1226.0,1322.0,737.0


In [20]:
flu_pivot.idxmax()

Year
2014    53
2015     1
2016    10
2017    52
2018     3
2019    11
2020    12
2021     2
dtype: int64

In [28]:
flu_pivot.mean(axis=1).sort_values(ascending=False).head()

Week
2     1640.750
3     1588.125
1     1572.875
53    1550.500
4     1486.500
dtype: float64

### Exercise 3: Top heart disease state

Perform a groupby-aggregation to compute the total number of "Disease of heart" by state. What state had the most heart disease deaths from 2014 through 2021?

In [31]:
deaths_df.groupby('State').agg({'Diseases of heart':'sum'}).head()

,Diseases of heart
State,
Alabama,106432.0
Alaska,6478.0
Arizona,101519.0
Arkansas,64735.0
California,498475.0


In [33]:
deaths_gb_state = deaths_df.groupby('State')

agg_d = {'Diseases of heart':'sum'}

deaths_gb_state.agg(agg_d).sort_values('Diseases of heart',ascending=False).head()

,Diseases of heart
State,
California,498475.0
Florida,379397.0
Texas,369248.0
Pennsylvania,258485.0
Ohio,228152.0


### Exercise 4: Annual mid-year "All Cause" deaths

Perform a groupby-aggregation on Year and Week to compute the total "All Cause" deaths. Use slicing to identify the number of "All Cause" deaths in the 26th week of each year.

In [41]:
agg_d = {'All Cause':'sum'}

deaths_all_cause_year_week_agg = deaths_df.groupby(['Year','Week']).agg(agg_d)

deaths_all_cause_year_week_agg

All Cause
Year Week           
2014 1       53523.0
     2       55743.0
     3       54241.0
     4       53759.0
     5       53450.0
...              ...
2021 48      63472.0
     49      62677.0
     50      56687.0
     51      45574.0
     52      32369.0

[418 rows x 1 columns]

In [46]:
deaths_all_cause_year_week_agg.loc[(2021,26)]

All Cause    56896.0
Name: (2021, 26), dtype: float64

In [40]:
deaths_all_cause_year_week_agg.loc[26]

,All Cause
Year,
2014,47131.0
2015,48798.0
2016,49348.0
2017,50229.0
2018,50403.0
2019,51826.0
2020,57986.0
2021,56896.0


In [45]:
idx = pd.IndexSlice

deaths_all_cause_year_week_agg.loc[idx[:,26],:]

,,All Cause
Year,Week,
2014,26,47131.0
2015,26,48798.0
2016,26,49348.0
2017,26,50229.0
2018,26,50403.0
2019,26,51826.0
2020,26,57986.0
2021,26,56896.0


### Exercise 5: Comparing Diabetes deaths in 2019 and 2020 between Utah and Colorado

Perform a groupby-aggregation on Year and State and compute the total "Diabetes mellitus" deaths. Use slicing to identify the number of diabetes deaths in 2019 and 2020 for Utah and Colorado.

In [48]:
agg_d = {'Diabetes mellitus':'sum'}

diabetes_agg = deaths_df.groupby(['Year','State']).agg(agg_d)

diabetes_agg

Diabetes mellitus
Year State                           
2014 Alabama                   1278.0
     Alaska                       0.0
     Arizona                   1990.0
     Arkansas                   819.0
     California                8397.0
...                               ...
2021 Virginia                  2509.0
     Washington                2141.0
     West Virginia              903.0
     Wisconsin                 1631.0
     Wyoming                      0.0

[416 rows x 1 columns]

In [50]:
diabetes_agg.loc[idx[[2019,2020],['Colorado','Utah']],:]

Diabetes mellitus
Year State                      
2019 Colorado             1051.0
     Utah                  631.0
2020 Colorado             1183.0
     Utah                  753.0

## Appendix

### Data preparation
Creating the deaths data by combing two files from the CDC for [2014-2019](https://data.cdc.gov/NCHS/Weekly-Counts-of-Deaths-by-State-and-Select-Causes/3yf8-kanr) and [2020-2022](https://data.cdc.gov/NCHS/Weekly-Provisional-Counts-of-Deaths-by-State-and-S/muzy-jte6).

In [ ]:
# We'll use regular expressions to clean up parts of the data
import re

# Read the data from the CDC website
deaths_2022_df = pd.read_csv(
    'https://data.cdc.gov/api/views/muzy-jte6/rows.csv?accessType=DOWNLOAD',
    parse_dates=['Week Ending Date']
)

# Remove columns
cols = [col for col in deaths_2022_df.columns if 'flag_' in col]
cols += ['Data As Of']
deaths_2022_df.drop(columns = cols,inplace=True)

# Rename columns
deaths_2022_df.rename(columns = {
    'Jurisdiction of Occurrence':'State',
    'MMWR Year':'Year',
    'MMWR Week':'Week',
    'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)':'Other',
    'COVID-19 (U071, Multiple Cause of Death)':'COVID-19, Multiple Cause',
    'COVID-19 (U071, Underlying Cause of Death)':'COVID-19, Underlying Cause',
    'Week Ending Date':'Week date'
}, inplace=True)

# Remove the US and NYC which aren't states
deaths_2022_df = deaths_2022_df[~deaths_2022_df['State'].isin(['United States','New York City'])]

# Get rid of codes
deaths_2022_df.columns = [re.sub('\s\(.+\)','',col) for col in deaths_2022_df.columns]

In [ ]:
# Read the data from the CDC website
deaths_2019_df = pd.read_csv(
    'https://data.cdc.gov/api/views/3yf8-kanr/rows.csv?accessType=DOWNLOAD',
    parse_dates=['Week Ending Date']
)

# Remove columns
cols = [col for col in deaths_2019_df.columns if 'flag_' in col]
deaths_2019_df.drop(columns = cols,inplace=True)

# Rename columns
deaths_2019_df.rename(columns = {
    'Jurisdiction of Occurrence':'State',
    'MMWR Year':'Year',
    'MMWR Week':'Week',
    'All  Cause':'All Cause',
    'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)':'Other',
    'Week Ending Date':'Week date'
}, inplace=True)

# Remove the US and NYC which aren't states
deaths_2019_df = deaths_2019_df[~deaths_2019_df['State'].isin(['United States','New York City'])]

# Get rid of codes
deaths_2019_df.columns = [re.sub('\s\(.+\)','',col) for col in deaths_2019_df.columns]

In [ ]:
# Concatenate both DataFrames together
deaths_df = pd.concat([deaths_2019_df,deaths_2022_df])

# Fille missing values with 0s
deaths_df = deaths_df.fillna(0)

# Sort data
deaths_df.sort_values(['State','Year','Week'],inplace=True)

# Write to disk
deaths_df.to_csv('CDC_deaths_2014_2022.csv',index=False,encoding='utf8')

### Working with `GroupBy` objects

***ULTRA IMPORTANT NOTE***: 99.999% of the time you will `aggregate` the data together in combination with `groupby`. This section is simply to show you what is happening under the hood when you perform a `groupby`: you are making baby DataFrames that share some value in common.

DataFrameGroupBy objects have a `groups` attribute that returns a dictionary. The keys in this dictionary are the years and the value is an array of row indices where the data corresponding to the value you grouped on. In our example, the keys are the years 2014 to 2021 and the values are the indices for rows of deaths data that happened in that year.

In [ ]:
deaths_gb_year.groups

If we access the 2014 key, we get this index back.

In [ ]:
deaths_gb_year.groups[2014]

Use this index to Boolean index the original DataFrame and we only get rows of data in the 2014 year.

In [ ]:
deaths_df.loc[deaths_gb_year.groups[2014]]

If you don't trust me, we can count up all the values of the "year" column in this groupby-ified baby DataFrame. There are 2756 values of the year "2014" corresponding to the 53 weeks \* 52 states of data for 2014 and no other years present.

In [ ]:
deaths_df.loc[deaths_gb_year.groups[2014],'Year'].value_counts()

You can also access a group using the `.get_group` method, instead of what we just did above. This is simply a third way to find the "baby DataFrame" that we are going to perform aggregations on (first was boolean indexing, second was locating the rows). Note the "Year" are all "2014".

In [ ]:
deaths_gb_year.get_group(2014)

***AGAIN I WANT TO EMPHASIZE***, you will likely never need to use these `groupby`-specific functions in a real-life data analysis situation: I'm simply showing what is happening under the hood to develop your intuitions that we are making baby DataFrames that share some value in common. If you are using groupby functions like `.groups` or `.get_group` in your analysis for anything besides debugging, you are most definitely not using groupby-aggregation appropriately: **99.999% of the time you will use `groupby` in combination with `aggregate`.**